In [18]:
import os
import json
import pandas as pd

from dotenv import load_dotenv

from model import BaseModelRunner, ChatModelRunner

In [16]:
from pathlib import Path

DATA_PATH_DOWNLOAD = Path('D:\projects\ML\ollama_python_chat\data')
DATA_PATH_DOWNLOAD.mkdir(parents=True, exist_ok=True)


In [17]:
def load_data_xl(path: Path, name) -> pd.DataFrame:
    print("Read data set from path {path}".format(path=path))
    df = pd.read_excel(path / name)
    return df

def load_data_csv(path: Path, name, sep='|') -> pd.DataFrame:
    print("Read data set from path {path}".format(path=path))
    df = pd.read_csv(path / name, sep=sep)
    return df

def print_text(df):
    for text in df['text']:
        print(text)

# Предварительные действия

In [ ]:
cat_df = load_data_xl(DATA_PATH_DOWNLOAD, 'address_70b_last.xlsx')
# cat_df = cat_df[cat_df['address']]
cat_df

# Чат с Ollama

In [ ]:
class OllamaBatchProcessor(BaseModelRunner):
    prompt_template_default = '''
Задача:
Найди в тексте адрес или муниципальный объект и верни его в формате, подходящем для поиска в Yandex. Если в тексте не указан адрес или объект, верни "None".
Формат ответа:
Напиши объект json. В ответе должен быть только сам объект, без дополнительных комментариев.
Формат:
[{{"result": "адрес или объект" или None }}]
Список текстов для обработки:
{texts}.
'''

    def __init__(self, df: pd.DataFrame, model_name='llama3.1', model_dir=os.getcwd(), template=None, target_column_name='text', result_column_name="llama_result"):
        super().__init__(model_name=model_name, model_dir=model_dir)
        self.df = df
        self.prompt_template = template if template else self.prompt_template_default
        self.target_column_name = target_column_name
        self.result_column_name = result_column_name

    def _generate_prompt(self, batch_df: pd.DataFrame) -> str:
        texts = '\n'.join([f"index {index}; text: {row[self.target_column_name]}" for index, row in batch_df.iterrows()])
        prompt = self.prompt_template.format(texts=texts)
        return prompt

    def process_single_row(self, max_retries: int = 5) -> pd.DataFrame:
        for index, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Processing rows"):
            success = False
            for attempt in range(max_retries):
                try:
                    row_df = pd.DataFrame([row])
                    prompt = self._generate_prompt(row_df)
                    response = self.generate(prompt)
                    response_json = json.loads(response)

                    if response_json:
                        response_item = response_json[0]
                        result = response_item['result']
                        self.df.at[index, self.result_column_name] = result

                    success = True
                    break
                except Exception as e:
                    print(f"Ошибка при обработке строки с индексом {index}, попытка {attempt + 1} из {max_retries}: {e}")

            if not success:
                print(f"Не удалось обработать строку с индексом {index} после {max_retries} попыток")
                
        return self.df

    def process_batch(self, batch_size: int = 10, max_retries: int = 5) -> pd.DataFrame:
        for start_idx in tqdm(range(0, len(self.df), batch_size), desc="Processing batches"):
            batch_df = self.df.iloc[start_idx:start_idx + batch_size]
            success = False
            for attempt in range(max_retries):
                try:
                    prompt = self._generate_prompt(batch_df)
                    response = self.generate(prompt)
                    response_json = json.loads(response)

                    if response_json:
                        for i, response_item in enumerate(response_json):
                            index = batch_df.index[i]
                            result = response_item['result']
                            self.df.at[index, self.result_column_name] = result

                    success = True
                    break
                except Exception as e:
                    print(f"Ошибка при обработке батча с индексом {start_idx}, попытка {attempt + 1} из {max_retries}: {e}")

            if not success:
                print(f"Не удалось обработать батч с индексом {start_idx} после {max_retries} попыток")
                
        return self.df


In [21]:
prompt = '''
Задача:
Найди в тексте точный адрес, территориальный объект (например школа или больница) или регион и верни его в формате, подходящем для поиска в Yandex. Если в тексте нет этого пиши "None". 
Дополнение:
Номера телефонов писать не надо. Область писать не надо. Некоторые регионы похожы на имена людей. Улицы должны быть в начальной форме. Если есть номер дома то указывай его. Если указано несколько отдельных адресов пиши их через ';'.
Формат ответа:
Напиши объект json, без дополнительных комментариев.
Формат:
[{{"result": "адрес или объект" }}]
Список текстов для обработки:
{texts}."
'''

In [22]:
# prompt = '''
# Задача:
# Найди в тексте точный адрес, территориальный объект (например школа или больница) или регион и верни его в формате, подходящем для поиска в Yandex. Если в тексте не указан адрес или объект, верни "None". 
# Дополнение:
# Номера телефонов писать не надо. Область писать не надо. Улицы должны быть в начальной форме. Если есть номер дома то указывай его. Если нет точного адреса пиши None.
# Формат ответа:
# Напиши объект json, без дополнительных комментариев. Ответ должен быть в одну строку.
# Формат:
# [{{"result": "адрес или объект или None" }}]
# Список текстов для обработки:
# {texts}."
# '''

In [ ]:
# work_df = cat_df.sample(frac=1)
# work_df = cat_df.sample(10)
work_df = cat_df.sample(10000)
load_dotenv()

# model_name = os.getenv('MODEL_NAME', 'llama3.1')
model_name = 'llama3.1:70b'
# model_name = 'llama3.1'
model_dir = os.getenv('OLLAMA_MODELS', os.getcwd())

processor = OllamaBatchProcessor(work_df, model_name=model_name, model_dir=model_dir, result_column_name='address_70B',target_column_name='text', template=prompt)

In [ ]:
result_df = processor.process_single_row()

In [11]:
pd.set_option('display.max_colwidth', 200)  

In [ ]:
result_df

In [ ]:
df_filtered = cat_df[~cat_df['id'].isin(result_df['id'])]
df_filtered.to_excel(DATA_PATH_DOWNLOAD / 'address_70b_last.xlsx', index=False)
df_filtered

In [14]:
result_df.to_excel(DATA_PATH_DOWNLOAD / 'address_70B_2.xlsx', index=False)